<div class="alert alert-success"><h1>Hyperparameter Tuning a Deep Learning Model in Python</h1></div>

Hyperparameter tuning is the process of systematically searching for the optimal combination of model parameters prior to training. These parameters, such as the number of hidden layers, the dropout rates between layers, the optimizer learning rate, and training batch size, play a critical role in determining the performance and generalizability of a deep learning model. Effective hyperparameter tuning can significantly improve model accuracy and robustness. In this tutorial, we will use the MNIST digit classification task to illustrate how to leverage Keras Tuner to discover the best configuration of hyperparameters for a deep learning model in Python.

## Learning Objectives
By the end of this tutorial, you will:
+ Understand the importance of hyperparameter tuning in deep learning.
+ Learn how to define and parameterize a tunable model using Keras Tuner.
+ Explore the use of Hyperband, an efficient tuning algorithm, to optimize hyperparameters like layer sizes, dropout rates, learning rates, and batch sizes.
+ Evaluate the performance of a tuned model on unseen data and compare it to a baseline model.


## Prerequisites
Before we begin, ensure you have:
+ Basic knowledge of Python programming (variables, functions, classes).
+ Familiarity with the fundamentals of how to build a deep learning model in Python using Keras.
+ A Python (version 3.x) environment with the `tensorflow`, `keras`, and `keras_tuner`packages installed.

<div class="alert alert-info"><b>Note:</b>To learn more about deep learning and how to build a deep learning model in Python using Keras, refer to  the LinkedIn Learning course titled <b>"Deep Learning with Python: Foundations"</b>.</div>

<div class="alert alert-success"><h2>1. Import and Preprocess the Data</h2></div>

We start by importing the data. For this tutorial, we'll use the **MNIST dataset**, a classic dataset in the machine learning community. It consists of 70,000 grayscale images of handwritten digits ranging from 0 to 9. Each image is 28 x 28 pixels, and the dataset is divided into 60,000 training images and 10,000 testing images. Our goal will be to develop a model that learns to correctly identify a handritten digit given the image.

In [ ]:
import tensorflow as tf
from tensorflow import keras

keras.utils.set_random_seed(1234)
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

Our deep learning model expects the images as a vector of size 784 (i.e. 28 $\times$ 28). So, let's flatten the images.

In [ ]:
train_images = train_images.reshape(60000, 28 * 28)
test_images = test_images.reshape(10000, 28 * 28)

The model also expects the image pixel values scaled. Let's do that as well.

In [ ]:
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

Finally, we also need to one-hot encode the image labels.

In [ ]:
num_classes = 10
train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

<div class="alert alert-success"><h2>2. Define the Baseline Model Architecture</h2></div>

The baseline model consists of an input layer with 784 nodes, two hidden layers with 32 and 16 nodes (respectively), and an output layer with 10 nodes.

In [ ]:
from tensorflow.keras.layers import Input, Dense

model = keras.Sequential([
    Input(shape = (784,)),
    Dense(32, activation = 'relu'),
    Dense(16, activation = 'relu'),
    Dense(10, activation = 'softmax')
])

<div class="alert alert-success"><h2>3. Train and Evaluate the Baseline Model</h2></div>

Next, we compile and train the baseline model.

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

history = model.fit(
    train_images, 
    train_labels,
    epochs = 10,
    validation_split = 0.1,
    batch_size = 128
)

Finally, we evaluate the model's performance against the test data.

In [ ]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

<div class="alert alert-success"><h2>4. Define the Tunable Model Architecture</h2></div>

Before we search for the optimal hyperparameters for our model, we need to define a function that specifies the architectural blueprint of the model. The blueprint will incorporate hyperparameters for the number of units per layer, dropout rates, and the optimizer learning rate. Keras Tuner will invoke this function multiple times with different hyperparameter values in order to find an optimal combination that maximizes validation accuracy.

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    model = keras.Sequential()
    model.add(Input(shape = (784,)))
    
    # Tune number of units for the first hidden layer
    model.add(Dense(units = hp.Int('hidden1', 32, 512, step = 32), activation = 'relu'))

    # Tune dropout rate after the first hidden layer
    model.add(Dropout(rate = hp.Float('dropout1', 0.1, 0.5, step = 0.1)))
    
    # Tune number of units for the second hidden layer
    model.add(Dense(units = hp.Int('hidden2', 16, 128, step = 16), activation = 'relu'))
    
    # Tune dropout rate after the second hidden layer
    model.add(Dropout(rate = hp.Float('dropout2', 0.1, 0.5, step = 0.1)))

    # Output layer with 10 units for the 10 classes
    model.add(Dense(units = 10, activation = 'softmax'))
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.0001, 0.001, or 0.01
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-4, 1e-3, 1e-2])

    
    # Compile the tunable model
    model.compile(
        optimizer = Adam(learning_rate = hp_learning_rate),
        loss = 'categorical_crossentropy',
        metrics = ['accuracy']
    )
    
    return model

<div class="alert alert-success"><h2>5. Running the Hyperparameter Search</h2></div>

Having defined our hyperparameter-tunable model, we now need to set up the tuner. Here, we choose **Hyperband**, a resource-efficient approach to hyperparameter tuning that builds upon random search and combines it with the principles of early stopping. Its primary goal is to reduce the computational cost of hyperparameter optimization by dynamically allocating more resources to promising hyperparameter configurations and fewer resources to less promising ones.

We will now start the search process with `tuner.search()`. This command will build and train multiple models using different combinations of hyperparameters. It will try different layer sizes, dropout rates, learning rates, and batch sizes, evaluating each combination’s performance on the validation set. 

Once the search is complete, we can output the best configuration of hyperparameters.

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(
    f"\nThe optimal number of units in the first and second densely-connected "
    f"layers are {best_hps.get('hidden1')} with a dropout rate of {best_hps.get('dropout1'):.2f} and "
    f"{best_hps.get('hidden2')} with a dropout rate of {best_hps.get('dropout2'):.2f}, respectively. "
    f"The optimal learning rate for the optimizer is {best_hps.get('learning_rate')} and "
    f"the optimal training batch size is {best_hps.get('batch_size')}."
)

Next, using the optimal set of hyperparameters, we create a tuned model.

In [ ]:
best_model = tuner.hypermodel.build(best_hps)

<div class="alert alert-success"><h2>6. Train and Evaluate the Tuned Model</h2></div>

Finally, we train the tuned model, ...

... and eveluate how well the tuned model generalizes to new data. 

In [ ]:
test_loss, test_accuracy = best_model.evaluate(test_images, test_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

This time, we see that the performance of the tuned model on the test data outperforms that of the baseline model. By systematically searching through a range of potential hyperparameters, the tuner identifies a configuration that better balances the complexity of the network with the demands of the data. This improved balance helps the model capture relevant patterns without overfitting, leading to higher accuracy on unseen examples. 

For many problems, careful hyperparameter tuning can yield substantial gains over default or intuitively chosen parameters. It also provides insights into which aspects of the model’s architecture and training process are most influential for performance, guiding future experimentation and development.